In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from cifar import load_CIFAR10
plt.rcParams['figure.figsize'] = (10.0, 8.0) 

cifar10_dir = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [3]:
X_train=X_train.reshape((X_train.shape[0], 32, 32, 3)).transpose(0,3,1,2)
X_test=X_test.reshape((X_test.shape[0], 32, 32, 3)).transpose(0,3,1,2)
print X_train.shape,X_test.shape

(50000, 3, 32, 32) (10000, 3, 32, 32)


In [4]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *
from lasagne.layers import Conv2DLayer as conv
from lasagne.layers import MaxPool2DLayer as pool
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu")
input_X = T.tensor4("X",dtype='float32')
target_y = T.vector("target Y integer",dtype='int32')

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN not available)
Using Theano backend.


In [ ]:
#input_layer = lasagne.layers.InputLayer(shape=(None,3, 32, 32), input_var=input_X)
#l_conv1 = lasagne.layers.Conv2DLayer(
#       input_layer, num_filters=32, filter_size=(5, 5),
#       nonlinearity=leaky_rectify)
#dense_output = lasagne.layers.DenseLayer(l_conv1,num_units = 10, nonlinearity=softmax)

Так задаётся архитектура нейронки

In [5]:
input_layer = lasagne.layers.InputLayer(shape=(None,3, 32, 32), input_var=input_X)
#nnet = <сверочная нейросеть>
# Для вдохновления 
# - http://torch.ch/blog/2015/07/30/cifar.html
# - http://www.robots.ox.ac.uk/~vgg/research/very_deep/
# - https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf
l_conv1 = lasagne.layers.Conv2DLayer(input_layer, 
            num_filters=32,
            filter_size=(5, 5),
            pad='same',
            nonlinearity=leaky_rectify)
l_conv1a = lasagne.layers.Conv2DLayer(l_conv1, 
            num_filters=32,
            filter_size=(5, 5),
            pad='same',
            nonlinearity=leaky_rectify)

l_maxpool1 = lasagne.layers.MaxPool2DLayer(l_conv1a, 
       pool_size=(3,3))

l_drop1=lasagne.layers.DropoutLayer(l_maxpool1,
        p=0.25)

l_conv2 = lasagne.layers.Conv2DLayer(
        l_drop1, num_filters=64, filter_size=(5, 5),
        nonlinearity=leaky_rectify)

l_conv2a = lasagne.layers.Conv2DLayer(
        l_conv2, num_filters=64, filter_size=(5, 5),
        nonlinearity=leaky_rectify)    

l_maxpool2 = lasagne.layers.MaxPool2DLayer(l_conv2a, 
        pool_size=2)

l_drop2=lasagne.layers.DropoutLayer(l_maxpool2,
        p=0.25)

l_dense1 = lasagne.layers.DenseLayer(l_drop2,
        num_units=512,
        nonlinearity=leaky_rectify)
l_drop3=lasagne.layers.DropoutLayer(l_dense1,
        p=0.5)
l_dense2 = lasagne.layers.DenseLayer(l_drop3,
        num_units=256,
        nonlinearity=leaky_rectify)
dense_output = lasagne.layers.DenseLayer(l_dense2,num_units = 10, nonlinearity=softmax)

In [6]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(dense_output)

/home/elyha7/anaconda2/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/elyha7/anaconda2/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/elyha7/anaconda2/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


In [7]:
#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(dense_output)
print all_weights

[W, b, W, b, W, b, W, b, W, b, W, b, W, b]


### дальше вы могли бы просто
* задать функцию ошибки вручную
* посчитать градиент ошибки по all_weights
* написать updates
* но это долго, а простой шаг по градиенту - не самый лучший смособ оптимизировать веса

Вместо этого, опять используем lasagne

In [8]:
loss = T.mean(T.nnet.categorical_crossentropy(y_predicted, target_y))
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()
updates_sgd = lasagne.updates.adam(loss, all_weights)

In [9]:
train_fun = theano.function([input_X,target_y],[loss, accuracy], updates=updates_sgd,allow_input_downcast=True)
accuracy_fun = theano.function([input_X,target_y],accuracy,allow_input_downcast=True)

### Вот и всё, пошли её учить

In [10]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# Процесс обучения

In [11]:
import time
from tqdm import tqdm

num_epochs = 170 #количество проходов по данным

batch_size = 500 #размер мини-батча

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in tqdm(iterate_minibatches(X_train, y_train,batch_size)):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    if epoch%10==0:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))

        print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
        print("  train accuracy:\t\t{:.2f} %".format(
            train_acc / train_batches * 100))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

100it [03:03,  1.81s/it]
0it [00:00, ?it/s]

Epoch 1 of 170 took 190.769s
  training loss (in-iteration):		2.341674
  train accuracy:		16.69 %
  validation accuracy:		25.09 %


100it [03:01,  1.81s/it]
100it [03:03,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:03,  1.86s/it]
0it [00:00, ?it/s]

Epoch 11 of 170 took 191.368s
  training loss (in-iteration):		0.996001
  train accuracy:		65.24 %
  validation accuracy:		61.95 %


100it [03:03,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:03,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
0it [00:00, ?it/s]

...

100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.82s/it]
100it [03:01,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
0it [00:00, ?it/s]

Epoch 151 of 170 took 188.558s
  training loss (in-iteration):		0.359556
  train accuracy:		87.77 %
  validation accuracy:		70.95 %


100it [03:01,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:00,  1.82s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
0it [00:00, ?it/s]

Epoch 161 of 170 took 188.551s
  training loss (in-iteration):		0.345758
  train accuracy:		88.49 %
  validation accuracy:		71.01 %


100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:01,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]
100it [03:00,  1.81s/it]


In [12]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 80:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"

Final results:
  test accuracy:		71.54 %
Нужно больше магии!


## Заполните форму

https://goo.gl/forms/FsANPB1jSqmX1JBJ3